# Наивный байесовский классификатор

$$P(y | x_1,x_2,...,x_n) = \frac{P(y) P(x_1,x_2,...,x_n|y)}{P(x_1,x_2,...,x_n)} $$

В силу 'наивного' предположения о независимости признаков $x_1,x_2,..,x_n$ получаем:
$$ P(y | x_1,x_2,...,x_n) = \frac{P(y) \prod\limits_{i=1}^{n}P(x_i| y)}{P(x_1,x_2,...,x_n)} $$

Откуда следует, что

$$ P(y | x_1,x_2,...,x_n) \propto P(y) \prod\limits_{i=1}^{n}P(x_i| y) $$

И тогда:

$$\hat{y} = arg \max_{y} P(y | x) $$

$$\hat{y} = arg \max_{y} P(y) \prod\limits_{i=1}^{n}P(x_i| y) $$

Поскольку мы ищем $argmax()$, то ничего не мешает нам логарифмировать выражение для более простого рассчета:

$$\hat{y} = arg \max_{y} (\ln(P(y)) + \sum\limits_{i=1}^{n} \ln(P(x_i| y)) ) $$

Вероятность $P(y)$ считается, исходя из частотных характеристик $y_{train}$: 
$$ P(y_{i}) = \frac{count(y_{i})}{count(y_{train})}$$

Но при рассчете условных вероятностей $P(x_i| y)$ будем учитывать 2 вещи. \
Во-первых: 
$$ P(x_i | y) = P(x_i =1 | y) x_i + (1 - P(x_i =1 | y)) (1 - x_i), $$
Во-вторых, каждая условная вероятность $ p_y = P(x_i =1 | y)$ считается с учетом аддитивного сглаживания (сглаживания Лапласса). \
Поскольку мы не должны допустить возможность деления на 0:
$$ p_y =  \frac{\alpha+\sum_{i=1}^n \mathbb{1}\{y_i = y\}x_{i,j}}{2\alpha + \sum_{i=1}^n\mathbb{1}\{y_i = y\}}$$

https://github.com/esokolov/ml-course-hse/blob/master/2021-spring/seminars/sem16-bayes.pdf

In [1]:
from sklearn.datasets import fetch_20newsgroups, make_classification
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, confusion_matrix
from sklearn.naive_bayes import BernoulliNB

import numpy as np
import pandas as pd
import scipy

In [2]:
%%time
# мы выбрали такой датасет, поскольку BernoulliNaiveBayes используется:
# - с положительными признаками (частотность слов всегда положительна)
# - с бинарными признаками (ну тут без этого :D )
data = fetch_20newsgroups(subset='all', data_home='data/')

CPU times: total: 156 ms
Wall time: 157 ms


In [3]:
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'],
                                                   test_size = 0.2,
                                                   stratify = data['target'],
                                                   random_state=42)

count = CountVectorizer(binary=True, lowercase=True, max_df=500, min_df=30)
X_train = count.fit_transform(X_train)
X_test = count.transform(X_test)

print('# training examples:', X_train.shape)
print('# test examples:', X_test.shape)

# training examples: (15076, 7649)
# test examples: (3770, 7649)


### Задание

Необходимо реализовать наивный байесовский классификтор для нормального распределения.
Сам код необходимо оформить и отправить боту в виде класса MyBernoulliNBClassifier в файле seminar03.py

Ваша реализация дожна поддерживать методы predict, predict_proba, score аналоично методам класса sklearn.naive_bayes.BernoulliNB при alpha=0

In [4]:
class MyBernoulliNBClassifier:
    def __init__(self, alpha=1, priors=None):
        self.alpha = alpha
    
    def fit(self, X, y):
        
        # кол-во встречаний каждого таргетаcount number of occurrences for each label
        y_counts = np.unique(y, return_counts=True)[1]
        self.classes_ = np.unique(y)
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
        
        # вычисляем P(y) - вероятность наблюдения каждого из таргетов
        class_prior = y_counts / y_counts.sum()
        self.log_class_prior = np.expand_dims(np.log(class_prior), axis = 1)
        
        # вычисляем P(x|y) - вероятность признака `х` при фикс таргете `у`
        prob_x_given_y = np.zeros([self.n_classes, self.n_features])
        
        for i in range(self.n_classes):
            
            # берем объекты из `Х` с выбранным таргетом
            row_mask = (y == i) 
            X_filtered = X[row_mask, :]
            
            # числитель: находим сумму по каждому признаку - `alpha` для сглаживания ( P(x and y) )
            numerator = (X_filtered.sum(axis = 0) + self.alpha)          # точно ли тут сумма, а не count?
            
            # знаменатель: количество объектов с таргетом `y` ( P(y) )
            denominator = (X_filtered.shape[0] + 2 * self.alpha)  
            prob_x_given_y[i, :] = numerator / denominator
        
        # вычисляем логарифм вероятностей для P(x|y) and P(~x|y)
        self.log_class_conditional_positive = np.log(prob_x_given_y)     # k x n matrix
        self.log_class_conditional_negative = np.log(1 - prob_x_given_y) # k x n matrix
    
    def _joint_likelihoods(self, X):
        if scipy.sparse.issparse(X):
            X = X.todense()                                                   # m x d matrix
        # вычисляем оба слагаемых в первом условии рассчета P(xi|y), затем находим его
        log_probs_positive = self.log_class_conditional_positive.dot(X.T)     # n x m matrix
        log_probs_negative = self.log_class_conditional_negative.dot(1 - X.T) # n x m matrix        
        log_likelihoods = log_probs_positive + log_probs_negative             # n x m matrix
        log_joint_likelihoods = (log_likelihoods + self.log_class_prior).T    # n x m matrix
        return np.asarray(log_joint_likelihoods)
        
    def predict(self, X):
        jll = self._joint_likelihoods(X)
        return self.classes_[np.argmax(jll, axis=1)]

    def predict_log_proba(self, X):
        jll = self._joint_likelihoods(X)
        # нормализауем забытым знаменателем P(x) = P(f_1, ..., f_n)
        log_prob_x = scipy.special.logsumexp(jll, axis=1)
        return jll - np.atleast_2d(log_prob_x).T

    def predict_proba(self, X):
        return np.exp(self.predict_log_proba(X))

    def score(self, X, y):
        return accuracy_score(y, self.predict(X))

In [5]:
# предсказания нашей модели
bnb = MyBernoulliNBClassifier(alpha=1)
bnb.fit(X_train, y_train)
y_pred = bnb.predict(X_test)
y_pred_proba = bnb.predict_proba(X_test)
y_pred_log_proba = bnb.predict_log_proba(X_test)

print('test accuracy:', bnb.score(X_test, y_test))
print('macro f1 score:', f1_score(y_test, y_pred, average='macro'))

test accuracy: 0.8010610079575596
macro f1 score: 0.8046815233397144


In [6]:
# предсказания склерн-модели
clf = BernoulliNB(alpha=1, force_alpha=True)
clf.fit(X_train, y_train)
check = clf.predict(X_test)
check_proba = clf.predict_proba(X_test)
check_log_proba = clf.predict_log_proba(X_test)

print('test accuracy:', clf.score(X_test, y_test))
print('macro f1 score:', f1_score(y_test, check, average='macro'))

test accuracy: 0.8010610079575596
macro f1 score: 0.8046815233397144


In [7]:
round_num = 7

display((y_pred != check).sum())
display((np.round(y_pred_proba, round_num) != np.round(check_proba, round_num)).sum())
display((np.round(y_pred_log_proba, round_num) != np.round(check_log_proba, round_num)).sum())

0

0

0